<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">✅ <strong>Importando Librerias</strong></h4>
</div>

In [1]:
import pandas as pd

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">✅ <strong>Importando DataSet</strong></h4>
</div>

In [2]:
df = pd.read_parquet('../data/processed/docutrans_iamc.parquet')
df.head()

,num_empr_trans,cod_lugar_descarga,tipdocempdoctra,cod_enti_matricula,cod_tip_docu_c0008,cnt_pes_brut_c0008,cnt_bulto_c0008,cnt_volumen_c0008,cnt_container,cod_puer_des_c0008,cod_natucarga,tpoemp_partic_cn,tamano_partic_cn,identi_partic_cn,rat_pes_bult_c0008,segmento_partic_cn,target
0,20259814210,20543083888,1511,,704,37060.0,12720.0,600.00,2.0,PECLL,12,39,01,02,2.913522,A,1
1,20507646051,20513462388,1505,54,714,6790.0,239.0,68.00,1.0,PECLL,12,02,03,01,28.410042,C,0
2,20507646051,20513462388,1505,54,714,6190.0,280.0,60.44,1.0,PECLL,12,07,03,02,22.107143,C,0
3,20507646051,20513462388,1505,54,714,20420.0,1143.0,68.00,1.0,PECLL,12,39,02,02,17.865267,C,0
4,20507646051,20513462388,1505,54,714,6336.0,206.0,28.00,1.0,PECLL,12,39,03,02,30.757282,C,0


<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">✅ <strong>Obteniendo Data Train y Test</strong></h4>
</div>

In [3]:
test_0 = df[df['target'] == 0].sample(frac=0.10)
test_1 = df[df['target'] == 1].sample(frac=0.10)

test = pd.concat([test_0, test_1])
train = df.drop(test.index)

test.to_parquet('../data/processed/test_15.parquet')
train.to_parquet('../data/processed/train_85.parquet')

del test_0, test_1, test, train

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">✅ <strong>Creando Transformacion de Variables Categoricas a Numericas</strong></h4>
</div>

In [4]:
train = pd.read_parquet('../data/processed/train_85.parquet')

variables_categoricas = train.select_dtypes(include=["object","category"]).columns.tolist()
print("Columnas categoricas: ",variables_categoricas) 

Columnas categoricas:  ['num_empr_trans', 'cod_lugar_descarga', 'tipdocempdoctra', 'cod_enti_matricula', 'cod_tip_docu_c0008', 'cod_puer_des_c0008', 'cod_natucarga', 'tpoemp_partic_cn', 'tamano_partic_cn', 'identi_partic_cn', 'segmento_partic_cn']


In [5]:
from category_encoders import CatBoostEncoder, TargetEncoder
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers = [ ('target_enc', TargetEncoder(), variables_categoricas) ],
    remainder='passthrough'  # deja las columnas numéricas como están
)

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">✅ <strong>Preparando Data para el Training</strong></h4>
</div>

In [6]:
X_train = train.drop('target', axis=1)
y_train = train['target']

test = pd.read_parquet('../data/processed/test_15.parquet')
X_test = test.drop('target', axis=1)
y_test = test['target']

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">
    ✅ <strong>Entrenando Modelos 
      <span style="color:#2e8b57; font-weight:bold; font-size:14px; background-color:#e0f8e0; padding:2px 4px; border-radius:3px;">
        sklearn.ensemble.RandomForestClassifier
      </span>
    </strong>
  </h4>
</div>


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import mlflow
from datetime import datetime
from sklearn.pipeline import Pipeline
import json

configuraciones = [
    {'n_estimators': 100, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': True, 'criterion': 'gini'},
    {'n_estimators': 200, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': False, 'criterion': 'entropy'},
    {'n_estimators': 150, 'max_depth': 20, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'bootstrap': True, 'criterion': 'gini'},
    {'n_estimators': 50,  'max_depth': 5,  'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': True, 'criterion': 'entropy'},
    {'n_estimators': 120, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': False, 'criterion': 'gini'},
    {'n_estimators': 300, 'max_depth': 15, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'bootstrap': True, 'criterion': 'entropy'},
    {'n_estimators': 80,  'max_depth': 8,  'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': True, 'criterion': 'gini'},
    {'n_estimators': 250, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': False, 'criterion': 'entropy'},
    {'n_estimators': 180, 'max_depth': 12, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': True, 'criterion': 'gini'},
    {'n_estimators': 100, 'max_depth': 6,  'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False, 'criterion': 'entropy'},
    {'n_estimators': 220, 'max_depth': 18, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': True, 'criterion': 'gini'},
    {'n_estimators': 90,  'max_depth': 7,  'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': False, 'criterion': 'entropy'},
    {'n_estimators': 160, 'max_depth': None, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': True, 'criterion': 'gini'},
    {'n_estimators': 130, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'bootstrap': True, 'criterion': 'entropy'},
    {'n_estimators': 275, 'max_depth': 14, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': False, 'criterion': 'gini'}
]

print("COMENZANDO A ENTRENAR MODELOS:")

for i, config in enumerate(configuraciones, start=1):

    model = RandomForestClassifier(**config)

    pipeline = Pipeline(
        [ ('preprocesamiento', preprocessor), ('clasificador', model) ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_proba = pipeline.predict_proba(X_test)[:, 1]

    # Evaluación
    accuracy = accuracy_score(y_test, y_pred)
    error_rate = 1 - accuracy
    model_type = type(pipeline.named_steps['clasificador']).__name__

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    xName = f"run_{timestamp}"
    config_str = json.dumps(config)  # sin indentación, todo en una sola línea

    with mlflow.start_run(run_name=xName):
        mlflow.log_param("1.Modelo_Usado", model_type)
        mlflow.log_param("2.Configuracion", config_str)
        mlflow.log_param("3.Transformacion", "category_encoders.TargetEncoder")
        mlflow.log_metric("1.error_rate", round(error_rate,5))
        mlflow.log_metric("2.accuracy", round(accuracy,5) )
        mlflow.sklearn.log_model(model, "modelo_random_forest")


COMENZANDO A ENTRENAR MODELOS:


2025/09/06 00:07:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run run_20250906_000705 at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0/runs/6a073a30b9754141bde1cc04169ce1b2
🧪 View experiment at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0


2025/09/06 00:07:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run run_20250906_000715 at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0/runs/a9ee7f02d65446aa8037bf61a300d473
🧪 View experiment at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0


2025/09/06 00:07:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run run_20250906_000727 at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0/runs/d5d00977b68f4fd08da84cf5fdae15b1
🧪 View experiment at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import mlflow
from datetime import datetime
import json

configuraciones = [
    {'penalty': 'l2', 'C': 1.0, 'solver': 'lbfgs', 'max_iter': 1000, 'fit_intercept': True},
    {'penalty': 'l1', 'C': 0.5, 'solver': 'liblinear', 'max_iter': 500, 'fit_intercept': True},
    {'penalty': 'l2', 'C': 0.1, 'solver': 'sag', 'max_iter': 1000, 'fit_intercept': False},
    {'penalty': 'elasticnet', 'C': 1.0, 'solver': 'saga', 'l1_ratio': 0.5, 'max_iter': 1000},
    {'penalty': 'l2', 'C': 2.0, 'solver': 'newton-cg', 'max_iter': 1500, 'fit_intercept': True},
    {'penalty': 'l1', 'C': 0.8, 'solver': 'liblinear', 'max_iter': 800, 'class_weight': 'balanced'},
    {'penalty': 'l2', 'C': 0.3, 'solver': 'lbfgs', 'max_iter': 1200, 'warm_start': True},
    {'penalty': 'elasticnet', 'C': 0.7, 'solver': 'saga', 'l1_ratio': 0.3, 'max_iter': 1000},
    {'penalty': 'l2', 'C': 1.5, 'solver': 'sag', 'max_iter': 900, 'n_jobs': -1},
    {'penalty': 'l1', 'C': 0.2, 'solver': 'liblinear', 'max_iter': 600, 'intercept_scaling': 2},
    {'penalty': 'l2', 'C': 0.05, 'solver': 'lbfgs', 'max_iter': 2000, 'class_weight': None},
    {'penalty': 'elasticnet', 'C': 1.2, 'solver': 'saga', 'l1_ratio': 0.7, 'max_iter': 1500},
    {'penalty': 'l2', 'C': 0.9, 'solver': 'newton-cg', 'max_iter': 1000, 'fit_intercept': False},
    {'penalty': 'l1', 'C': 1.0, 'solver': 'liblinear', 'max_iter': 1000, 'dual': False},
    {'penalty': 'l2', 'C': 0.6, 'solver': 'saga', 'max_iter': 1100, 'class_weight': 'balanced'}
]

print("COMENZANDO A ENTRENAR MODELOS:")

for i, config in enumerate(configuraciones, start=1):

    model = LogisticRegression(**config)

    pipeline = Pipeline(
        [ ('preprocesamiento', preprocessor), ('clasificador', model) ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_proba = pipeline.predict_proba(X_test)[:, 1]

    # Evaluación
    accuracy = accuracy_score(y_test, y_pred)
    error_rate = 1 - accuracy
    model_type = type(pipeline.named_steps['clasificador']).__name__

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    xName = f"run_{timestamp}"
    config_str = json.dumps(config)  # sin indentación, todo en una sola línea

    with mlflow.start_run(run_name=xName):
        mlflow.log_param("1.Modelo_Usado", model_type)
        mlflow.log_param("2.Configuracion", config_str)
        mlflow.log_param("3.Transformacion", "category_encoders.TargetEncoder")
        mlflow.log_metric("1.error_rate", round(error_rate,5))
        mlflow.log_metric("2.accuracy", round(accuracy,5) )
        mlflow.sklearn.log_model(model, "modelo_random_forest")


COMENZANDO A ENTRENAR MODELOS:


2025/09/06 00:11:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run run_20250906_001144 at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0/runs/b6e4c7859e8f41c29f73e59a634ea3ad
🧪 View experiment at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0


2025/09/06 00:11:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run run_20250906_001150 at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0/runs/9ada3c9469db47edac2f1423d94faa25
🧪 View experiment at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0


...

🏃 View run run_20250906_001331 at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0/runs/8c9681f52421454db15e0655081f7d1e
🧪 View experiment at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0


2025/09/06 00:13:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run run_20250906_001337 at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0/runs/556dbd6afd254ff1b6cc00e10fbc8fa3
🧪 View experiment at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0


/opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
2025/09/06 00:13:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run run_20250906_001347 at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0/runs/1debeb6193304bc2b0ea185e7db6e593
🧪 View experiment at: http://mlflow.default.svc.cluster.local:5000/#/experiments/0
